<a href="https://colab.research.google.com/github/ChandrashekharGhanokar/genai_finance_chatbot/blob/main/genai_finance_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Assuming the file is named 'BankFAQs.csv'
bank = pd.read_csv("/content/BankFAQs.csv")
bank.head()

,Question,Answer,Class
0,Do I need to enter ‘#’ after keying in my Card...,Please listen to the recorded message and foll...,security
1,What details are required when I want to perfo...,"To perform a secure IVR transaction, you will ...",security
2,How should I get the IVR Password if I hold a...,An IVR password can be requested only from the...,security
3,How do I register my Mobile number for IVR Pas...,Please call our Customer Service Centre and en...,security
4,How can I obtain an IVR Password,By Sending SMS request: Send an SMS 'PWD<space...,security


In [3]:
# Combine the Question and Answer columns into a single content column
bank["content"] = bank.apply(lambda row: f"Question: {row['Question']}\nAnswer: {row['Answer']}", axis=1)
bank.head()

,Question,Answer,Class,content
0,Do I need to enter ‘#’ after keying in my Card...,Please listen to the recorded message and foll...,security,Question: Do I need to enter ‘#’ after keying ...
1,What details are required when I want to perfo...,"To perform a secure IVR transaction, you will ...",security,Question: What details are required when I wan...
2,How should I get the IVR Password if I hold a...,An IVR password can be requested only from the...,security,Question: How should I get the IVR Password i...
3,How do I register my Mobile number for IVR Pas...,Please call our Customer Service Centre and en...,security,Question: How do I register my Mobile number f...
4,How can I obtain an IVR Password,By Sending SMS request: Send an SMS 'PWD<space...,security,Question: How can I obtain an IVR Password \nA...


In [4]:
!pip install langchain langchain_community sentence-transformers chromadb bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:0

In [5]:
from langchain.docstore.document import Document

# Prepare documents for LangChain
documents = []
for _, row in bank.iterrows():
    documents.append(Document(page_content=row["content"], metadata={"class": row["Class"]}))
documents[1]

Document(metadata={'class': 'security'}, page_content='Question: What details are required when I want to perform a secure IVR transaction\nAnswer: To perform a secure IVR transaction, you will need your 16-digit Card number, Card expiry date, CVV number, mobile number and IVR password.')

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

hg_embeddings = HuggingFaceEmbeddings()
persist_directory = '/content/'

langchain_chroma = Chroma.from_documents(
    documents=documents,
    collection_name="chatbot_finance",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)

<ipython-input-6-4b26b7621dfc>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  hg_embeddings = HuggingFaceEmbeddings()
<ipython-input-6-4b26b7621dfc>:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hg_embeddings = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer

model_id = 'HuggingFaceH4/zephyr-7b-beta'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Quantization configuration
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [8]:
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=6000,
    max_new_tokens=500,
    device_map="auto",
)

In [11]:
from IPython.display import display, Markdown
from langchain.llms import HuggingFacePipeline  # Make sure to import HuggingFacePipeline

def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Initialize the HuggingFacePipeline as the LLM
llm = HuggingFacePipeline(pipeline=query_pipeline)

# Define the question
question = "What is Chatbot and How is it used in Finance Domain?"

# Generate the response using the LLM
response = llm(question)

# The response is typically a list of generated texts, so extract the first item
generated_text = response[0]['generated_text'] if isinstance(response, list) else response

# Combine the question and generated answer for display
full_response =  f"Question: {question}\nAnswer: {generated_text}"
display(Markdown(colorize_text(full_response)))


<ipython-input-11-a3763e3abbd5>:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=query_pipeline)
<ipython-input-11-a3763e3abbd5>:16: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm(question)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific str



**<font color='red'>Question:</font>** What is Chatbot and How is it used in Finance Domain?


**<font color='green'>Answer:</font>** What is Chatbot and How is it used in Finance Domain?

A chatbot is an AI-powered software application that can simulate a conversation with human users through text or voice commands. It uses natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots are widely used in various industries, including finance, to provide quick and efficient customer service, automate routine tasks, and enhance the overall customer experience.

In the finance domain, chatbots are used to provide a range of services, including account management, transaction processing, investment advice, and financial education. They can help customers with tasks such as checking their account balance, transferring funds, paying bills, and making investments. Chatbots can also provide personalized financial advice based on the user's financial goals, risk tolerance, and investment horizon.

The use of chatbots in finance has several benefits, including:

1. 24/7 Availability: Chatbots are available 24/7, which means customers can access financial services at any time, without the need for human intervention.

2. Faster Response Time: Chatbots can respond to customer queries instantly, which reduces wait times and improves the overall customer experience.

3. Cost-Effective: Chatbots can automate routine tasks, which reduces the workload on human customer service representatives and lowers operational costs.

4. Personalized Experience: Chatbots can provide personalized financial advice based on the user's financial goals, risk tolerance, and investment horizon.

5. Enhanced Security: Chatbots can provide secure financial services by implementing advanced security measures, such as biometric authentication and encryption.

In this report, we will analyze the current state of chatbot adoption in the finance domain, the challenges and opportunities associated with chatbot implementation, and the future trends and predictions in this area.

Current State of Chatbot Adoption in Finance

The adoption of chatbots in finance has been growing rapidly in recent years. According to a report by Juniper Research, the number of chatbot users in the finance sector is expected to reach 1.5 billion by 2023, up from 115 million in 2018. This represents a compound annual growth rate (CAGR) of 34% over the next five years.

The report also found that the